In [14]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from torch.utils.data import Dataset

In [10]:
df = pd.read_csv("/work/diabetes.csv")
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [18]:
le = LabelEncoder()
y = np.array(le.fit_transform(y), dtype="float64")

In [33]:
sc = StandardScaler()
X = torch.tensor(sc.fit_transform(X))
y = torch.tensor(y)
y = torch.unsqueeze(y, 1)

/tmp/ipykernel_77/1798620586.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y)


In [51]:
class Dataset(Dataset):

    def __init__(self,x,y):
        self.x = x
        self.y = y
        
    def __getitem__(self,index):
        # Get one item from the dataset
        return self.x[index], self.y[index]
    
    def __len__(self):
        return len(self.x)

In [52]:
dataset = Dataset(X, y)

In [53]:
train_loader = torch.utils.data.DataLoader(dataset = dataset, batch_size = 32, shuffle = True)

In [59]:
class Model(nn.Module):
    def __init__(self, input_features):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_features, 5)
        self.fc2 = nn.Linear(5, 4)
        self.fc3 = nn.Linear(4, 3)
        self.fc4 = nn.Linear(3, 1)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
    
    def forward(self, X):
        out = self.fc1(X)
        out = self.tanh(out)
        out = self.fc2(out)
        out = self.tanh(out)
        out = self.fc3(out)
        out = self.tanh(out)
        out = self.fc4(out)
        out = self.sigmoid(out)
        return out


In [63]:
net = Model(X.shape[1])
criterion = torch.nn.BCELoss(size_average = True)
optimizer = torch.optim.SGD(net.parameters(), lr = 0.1, momentum = 0.9)

/shared-libs/python3.9/py/lib/python3.9/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [64]:
num_epochs = 200
for epoch in range(num_epochs):
    for inputs,labels in train_loader:
        inputs = inputs.float()
        labels = labels.float()
        # Feed Forward
        output = net(inputs)
        # Loss Calculation
        loss = criterion(output, labels)
        # Clear the gradient buffer (we don't want to accumulate gradients)
        optimizer.zero_grad()
        # Backpropagation 
        loss.backward()
        # Weight Update: w <-- w - lr * gradient
        optimizer.step()
        
    #Accuracy
    # Since we are using a sigmoid, we will need to perform some thresholding
    output = (output>0.5).float()
    # Accuracy: (output == labels).float().sum() / output.shape[0]
    accuracy = (output == labels).float().mean()
    # Print statistics 
    print("Epoch {}/{}, Loss: {:.3f}, Accuracy: {:.3f}".format(epoch+1,num_epochs, loss, accuracy))

Epoch 1/200, Loss: 0.544, Accuracy: 0.688
Epoch 2/200, Loss: 0.432, Accuracy: 0.844
Epoch 3/200, Loss: 0.435, Accuracy: 0.812
Epoch 4/200, Loss: 0.560, Accuracy: 0.625
Epoch 5/200, Loss: 0.408, Accuracy: 0.875
Epoch 6/200, Loss: 0.526, Accuracy: 0.750
Epoch 7/200, Loss: 0.407, Accuracy: 0.812
Epoch 8/200, Loss: 0.584, Accuracy: 0.750
Epoch 9/200, Loss: 0.379, Accuracy: 0.781
Epoch 10/200, Loss: 0.455, Accuracy: 0.781
Epoch 11/200, Loss: 0.531, Accuracy: 0.781
Epoch 12/200, Loss: 0.429, Accuracy: 0.844
Epoch 13/200, Loss: 0.515, Accuracy: 0.750
Epoch 14/200, Loss: 0.504, Accuracy: 0.812
Epoch 15/200, Loss: 0.426, Accuracy: 0.812
Epoch 16/200, Loss: 0.498, Accuracy: 0.781
Epoch 17/200, Loss: 0.479, Accuracy: 0.750
Epoch 18/200, Loss: 0.444, Accuracy: 0.781
Epoch 19/200, Loss: 0.464, Accuracy: 0.812
Epoch 20/200, Loss: 0.329, Accuracy: 0.812
Epoch 21/200, Loss: 0.586, Accuracy: 0.750
Epoch 22/200, Loss: 0.422, Accuracy: 0.750
Epoch 23/200, Loss: 0.392, Accuracy: 0.844
Epoch 24/200, Loss: 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=49da15f7-246c-45df-bc8f-6f9233c2b1e1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>